# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np

In [2]:
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=2020")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
def insert_from_dataframe(df, insert_query):
    for i, row in df.iterrows():
        cur.execute(insert_query, list(row))
    conn.commit()

In [5]:
filepath = 'data/song_data/'
for root, dirs, files in os.walk(filepath):
    print(f'root: {root} ')
    print(f'dirs: {dirs}')
    print(f'files: {files}')
    print('-'*100)

root: data/song_data/ 
dirs: ['A']
files: []
----------------------------------------------------------------------------------------------------
root: data/song_data/A 
dirs: ['A', 'B']
files: []
----------------------------------------------------------------------------------------------------
root: data/song_data/A\A 
dirs: ['A', 'B', 'C']
files: []
----------------------------------------------------------------------------------------------------
root: data/song_data/A\A\A 
dirs: []
files: ['TRAAAAW128F429D538.json', 'TRAAABD128F429CF47.json', 'TRAAADZ128F9348C2E.json', 'TRAAAEF128F4273421.json', 'TRAAAFD128F92F423A.json', 'TRAAAMO128F1481E7F.json', 'TRAAAMQ128F1460CD3.json', 'TRAAAPK128E0786D96.json', 'TRAAARJ128F9320760.json', 'TRAAAVG12903CFA543.json']
----------------------------------------------------------------------------------------------------
root: data/song_data/A\A\B 
dirs: []
files: ['TRAABCL128F4286650.json', 'TRAABDL12903CAABBA.json', 'TRAABJL12903CDCF1A.json', '

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data/')

In [7]:
filepath = song_files[0]
print(filepath)

D:\OneDrive - University Of Houston\Notebook_OneDrive\DE_udacity\Project_Data_Modeling_with_Postgres\data\song_data\A\A\A\TRAAAAW128F429D538.json


In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [9]:
df.values[0]

array([1, 'ARD7TVE1187B99BFB1', nan, nan, 'California - LA', 'Casual',
       'SOMZWCG12A8C13C480', "I Didn't Mean To", 218.93179, 0],
      dtype=object)

**Note**: The first file contains NaN, so it cannot be inserted into table, I will read all song files to df, and drop all rows which contain NaN.

In [10]:
print(f'0: {song_files[0]}')
print(f'-1: {song_files[-1]}')

0: D:\OneDrive - University Of Houston\Notebook_OneDrive\DE_udacity\Project_Data_Modeling_with_Postgres\data\song_data\A\A\A\TRAAAAW128F429D538.json
-1: D:\OneDrive - University Of Houston\Notebook_OneDrive\DE_udacity\Project_Data_Modeling_with_Postgres\data\song_data\A\B\C\TRABCUQ128E0783E2B.json


In [11]:
df_song = df #the first file
for idx in range(1, len(song_files)): #second to last file
    df_song = df_song.append(pd.read_json(song_files[idx], lines=True), ignore_index=True)
df_song.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


**Note**: in `sql_queries.py`, artist_id in `songs` table refer to artist_id in table `artists`, so the table `artists` must be inserted firstly.

## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
df_song.info()
artist_data = df_song[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         60 non-null     int64  
 1   artist_id         60 non-null     object 
 2   artist_latitude   26 non-null     float64
 3   artist_longitude  26 non-null     float64
 4   artist_location   60 non-null     object 
 5   artist_name       60 non-null     object 
 6   song_id           60 non-null     object 
 7   title             60 non-null     object 
 8   duration          60 non-null     float64
 9   year              60 non-null     int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 4.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         60 non-null     object 
 1   artist_name       60 no

In [13]:
artist_data = artist_data.drop_duplicates()
# artist_data = artist_data.replace(np.nan, None, regex=True)
artist_data = artist_data.replace({np.nan: None})
artist_data.columns = ['artist_id', 'name', 'location', 'latitude', 'longitude']
artist_data.head()

,artist_id,name,location,latitude,longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,None,None
1,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
2,ARKRRTF1187B9984DA,Sonora Santanera,,None,None
3,AR7G5I41187FB4CE6C,Adam Ant,"London, England",None,None
4,ARXR32B1187FB57099,Gob,,None,None


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
artist_table_insert

'\n    INSERT INTO artists (\n        artist_id,\n        name,\n        location,\n        latitude,\n        longitude\n    )\n    VALUES (%s, %s, %s, %s, %s)\n    ON CONFLICT (artist_id)\n    DO NOTHING\n'

In [15]:
insert_from_dataframe(artist_data, artist_table_insert)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [16]:
song_data = df_song[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_data.drop_duplicates()
song_data = song_data.replace({np.nan: None})
song_data.head()

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [17]:
insert_from_dataframe(song_data, song_table_insert)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files('data/log_data/')

In [19]:
filepath = log_files[0]
print(filepath)

D:\OneDrive - University Of Houston\Notebook_OneDrive\DE_udacity\Project_Data_Modeling_with_Postgres\data\log_data\2018\11\2018-11-01-events.json


In [20]:
df = pd.read_json(filepath, lines=True)
df_log = df
for idx in range(1, len(log_files)):
    df_log = df_log.append(pd.read_json(log_files[idx], lines=True), ignore_index=True)
print(f'df_log size: {df_log.shape}')
df_log.head()

df_log size: (8056, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
df_log = df_log[df_log['page']=='NextSong']
df_log = df_log.replace(np.nan, None)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [22]:
tf = pd.DataFrame({'start_time': pd.to_datetime(df_log['ts'], unit='ms')})

tf['hour'] = tf['start_time'].dt.hour
tf['day'] = tf['start_time'].dt.day
# tf['week'] = tf['start_time'].dt.week #deprecated
tf['week'] = tf['start_time'].dt.isocalendar().week
tf['month'] = tf['start_time'].dt.month
tf['year'] = tf['start_time'].dt.year
tf['weekday'] = tf['start_time'].dt.weekday

tf = tf.drop_duplicates()
tf.head()

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
insert_from_dataframe(tf, time_table_insert)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates()
user_df = user_df[user_df['userId'] != '']
user_df.columns = ['user_id', 'first_name', 'last_name', 'gender', 'level']
user_df.head()

,user_id,first_name,last_name,gender,level
2,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free
17,83,Stefany,White,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
insert_from_dataframe(user_df, user_table_insert)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # (songplay_id int, start_time int, user_id int, level text, song_id text, artist_id text, session_id int, location text, user_agent text)
    # songplay_data = (index, row.ts, int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.